In this notebook, we are going to create a Pascal VOC (kinda) version of Calcium OCT (from Sohee's)

The goal is for MMSeg to be able to read and use. Subsequently, RADIO and DINOv2 can use it for their linear probing semantic segmentaion of their feature map.

In [1]:
from pathlib import Path
import nibabel as nib
import numpy as np
import cv2

def get_color_map():
    rbg_map = np.array([
        [0, 0, 0], # background
        [1, 1, 1], # calcium
    ])
    label_map = [
        'background',
        'calcium'
    ]

    with open(output_root_dir / 'colors.txt', 'w') as f:
        for i in range(rbg_map.shape[0]):
            f.write(f"{label_map[i]} {rbg_map[i][0]} {rbg_map[i][1]} {rbg_map[i][2]}\n")
    return rbg_map

def save_image(image, path):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(str(path), image)

def load_nii(path):
    return nib.load(path).get_fdata()

def prepare_folder_structure(output_root_dir):
    output_root_dir = Path(output_root_dir)
    output_root_dir.mkdir(parents=True, exist_ok=True)

    image_dir = output_root_dir / "JPEGImages"
    segmentation_dir = output_root_dir / "SegmentationClass"
    imagesets_dir = output_root_dir / "ImageSets"

    image_dir.mkdir(exist_ok=False)
    
    segmentation_dir.mkdir(exist_ok=True)
    imagesets_dir.mkdir(exist_ok=True)

    return output_root_dir, image_dir, segmentation_dir, imagesets_dir

output_root_dir, image_dir, segmentation_dir, imagesets_dir = prepare_folder_structure(output_root_dir = "/storage_bizon/naravich/datasets/CalciumOCT_VOC")

nii_image_dir = Path("/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset302_Calcium_OCTv2/imagesTr/")
nii_segmentation_dir = Path("/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset302_Calcium_OCTv2/labelsTr/")

In [2]:
rbg_map = get_color_map()
count = 0
for nii_image_path in nii_image_dir.glob("*.nii.gz"):
    imag_name = nii_image_path.name[:-len('_0000.nii.gz')]
    label_name = imag_name + ".nii.gz"
    nii_segmentation_path = nii_segmentation_dir / label_name

    image = load_nii(nii_image_path)
    image = image.astype(np.float32) * 255
    image = 2 * image
    assert image.max() <= 255
    segmentation = load_nii(nii_segmentation_path)

    num_images = image.shape[2] # slice in z-axis
    for i in range(num_images):
        image_slice = image[:, :, i]
        segmentation_slice = segmentation[:, :, i]
        
        rgb_image = cv2.cvtColor(image_slice, cv2.COLOR_GRAY2RGB)

        rgb_segmentation = np.zeros((segmentation_slice.shape[0], segmentation_slice.shape[1], 3), dtype=np.uint8)
        for j in range(rbg_map.shape[0]):
            rgb_segmentation[segmentation_slice == j] = rbg_map[j]

        if segmentation_slice.max() > 0:
            save_image(rgb_image, image_dir / f"{imag_name}_{i}.jpg")
            save_image(rgb_segmentation, segmentation_dir / f"{imag_name}_{i}.png")
            with open(imagesets_dir / "train.txt", "a") as f:
                f.write(f"{imag_name}_{i}\n")
            if count > 10:
                break
            count += 1
            
    break